In [ ]:
#!pip install hanlp
#!pip install simalign

In [1]:
import os, csv

import pandas as pd
from tqdm import tqdm
from simalign import SentenceAligner
# from google.cloud import translate_v2 as translate

/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [178]:

src_lan = 'en'
# target language
tgt_lan = 'IT'
language = "Italian"

RESOURCE = '/Users/jairiley/Desktop/Research/Sense-Projection'
DATA =  RESOURCE+'/data/'+language+'/second-sense'
TRANS_FILE = os.path.join(DATA, 'gold-sentences-'+language+'.tsv')
TOKENS_FILE = os.path.join(DATA, 'tokens-gold-'+language+'.tsv')
ALIGN_SENT_FILE = os.path.join(DATA, 'sentences-aligned-'+language+'.tsv')
ALIGN_TOKEN_FILE = os.path.join(DATA, 'tokens-aligned-'+language+'.tsv')

# DATA = os.path.join(RESOURCE, 'data')
DATA =  RESOURCE+'/data/English/second-sense'

KEY = os.path.join(RESOURCE, 'key')
RAW_SENSE_TSV = os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
RAW_TOKENS_TSV =  os.path.join(DATA, 'tokens-gold-English.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'gold-sentences-English.tsv')
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, 'tonal-works-420505-eda807c7cc52.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS

# DATA

In [179]:
# raw
raw_sense_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
raw_sense_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,This,NaN,X,NaN
1,d000.s000.t000,document,document,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...
2,NaN,is,be,VERB,NaN
3,NaN,a,NaN,X,NaN
4,d000.s000.t001,summary,summary,NOUN,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...


In [180]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t',  quoting=csv.QUOTE_NONE)
sentence_df.head()
sents = sentence_df["ID"].tolist()
len(sents)
# print(sents)

121

In [181]:
sids = sentence_df["ID"].tolist()
# sids = [i.replace('d', 'i') for i in sids]
# sids[:4]
print(sids)

['d000.s000', 'd000.s001', 'd000.s002', 'd000.s003', 'd000.s004', 'd000.s005', 'd000.s006', 'd000.s007', 'd000.s008', 'd000.s009', 'd000.s010', 'd000.s011', 'd000.s012', 'd000.s013', 'd000.s014', 'd000.s015', 'd000.s016', 'd000.s017', 'd000.s018', 'd000.s019', 'd000.s020', 'd000.s021', 'd000.s022', 'd000.s023', 'd000.s024', 'd000.s025', 'd000.s026', 'd000.s027', 'd000.s028', 'd000.s029', 'd000.s030', 'd000.s031', 'd000.s032', 'd000.s033', 'd000.s034', 'd000.s035', 'd000.s036', 'd000.s037', 'd000.s038', 'd000.s039', 'd000.s040', 'd000.s041', 'd000.s042', 'd000.s043', 'd000.s044', 'd000.s045', 'd000.s046', 'd000.s047', 'd000.s048', 'd000.s049', 'd000.s050', 'd000.s051', 'd000.s052', 'd000.s053', 'd000.s054', 'd000.s055', 'd000.s056', 'd000.s057', 'd000.s058', 'd000.s059', 'd000.s060', 'd000.s061', 'd000.s062', 'd000.s063', 'd000.s064', 'd000.s065', 'd000.s066', 'd000.s067', 'd000.s068', 'd000.s069', 'd000.s070', 'd000.s071', 'd000.s072', 'd000.s073', 'd000.s074', 'd000.s075', 'd000.s076'

In [182]:
trans_df = pd.read_csv(TRANS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
trans_df.head()

,ID,Sentence
0,d000.s000,Questo documento è la sintesi di una relazione...
1,d000.s001,L EPAR descrive il modo in cui il comitato per...
2,d000.s002,Per maggiori informazioni riguardanti le propr...
3,d000.s003,Per maggiori informazioni riguardo alle motiva...
4,d000.s004,Che cos è Alimta ?


In [183]:
tkns_df = pd.read_csv(TOKENS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
tkns_df.head(1000)

,Token ID,Token,Lemma,POS
0,NaN,Questo,NaN,X
1,d000.s000.t000,documento,documento,NOUN
2,NaN,è,essere,VERB
3,NaN,la,NaN,X
4,d000.s000.t001,sintesi,sintesi,NOUN
...,...,...,...,...
995,d000.s038.t000,Non,non,ADV
996,NaN,è,essere,VERB
997,d000.s038.t001,necessario,necessario,ADJ
998,d000.s038.t002,conoscere,conoscere,VERB


In [184]:
tgt_tkns = tkns_df['Token'].tolist()
len(tgt_tkns)

2851

# ALIGNMENT

In [185]:
device = 'cpu'
layer = 8
# "bert": "bert-base-multilingual-cased",
# "xlmr": "xlm-roberta-base"
model = 'xlmr'
# mwmf (Match), inter (ArgMax), itermax (IterMax)
mode = 'inter'
matching_methods = 'mai'
token_type='bpe'
aligner = SentenceAligner(
    model=model
    , token_type=token_type
    , matching_methods=matching_methods
    , layer=layer
    , device=device
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-08-12 22:32:16,266 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base


In [186]:
raw_sense_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,This,NaN,X,NaN
1,d000.s000.t000,document,document,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...
2,NaN,is,be,VERB,NaN
3,NaN,a,NaN,X,NaN
4,d000.s000.t001,summary,summary,NOUN,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...


In [187]:
# english token sequences
idxes = raw_sense_df['Token ID'].tolist()
tokens = raw_sense_df['Token'].tolist()
# print(idxes,tokens)
en_sents = []
# en_sents2 = trans_df["English"].tolist()
# pre_sid = None
sent = []
for i, s in zip(idxes, tokens):
    if s != "-end-":
        sent.append(s)
    else:
        en_sents.append(sent)
        sent = []
print(len(en_sents))
# for x in range(len(en_sents)):
#     print(" ".join(en_sents[x]),en_sents2[x])
#     if " ".join(en_sents[x]) != en_sents2[x]:
#         print(x)

121


In [188]:
print(en_sents[0])

['This', 'document', 'is', 'a', 'summary', 'of', 'the', 'European', 'Public', 'Assessment', 'Report', '(', 'EPAR', ')', '.']


In [189]:
# Italian token sequences
idxes = tkns_df['Token ID'].tolist()
tokens = tkns_df['Token'].tolist()

tgt_sents = []
# en_sents2 = trans_df["English"].tolist()
# pre_sid = None
sent = []
for i, s in zip(idxes, tokens):
    if s != "-end-":
        sent.append(s)
    else:
        tgt_sents.append(sent)
        sent = []
# print(tgt_sents)
# tgt_sents = []
# pre_sid = None
# s = []
# for i, t in zip(idxes, tokens):
#     print(i)
#     sid = i.split('.')[1]
#     if sid != pre_sid:
#         if s:
#             tgt_sents.append(s)
#         pre_sid = sid
#         s = [t]
#     else:
#         s.append(t)
# tgt_sents.append(s)
print(tgt_sents)
print(len(tgt_sents))

[['Questo', 'documento', 'è', 'la', 'sintesi', 'di', 'una', 'relazione', 'pubblica', 'di', 'valutazione', 'europea', '(', 'EPAR', ')', '.'], ['L', 'EPAR', 'descrive', 'il', 'modo', 'in', 'cui', 'il', 'comitato', 'per', 'i', 'medicinali', 'per', 'uso', 'umano', '(', 'CHMP', ')', 'ha', 'valutato', 'gli', 'studi', 'scientifici', 'effettuati', 'e', 'ha', 'formulato', 'le', 'raccomandazioni', 'su', 'come', 'usare', 'il', 'medicinale', '.'], ['Per', 'maggiori', 'informazioni', 'riguardanti', 'le', 'proprie', 'condizioni', 'di', 'salute', 'o', 'la', 'terapia', ',', 'leggere', 'il', 'foglio', 'illustrativo', '(', 'allegato', 'all', 'EPAR', ')', 'oppure', 'consultare', 'il', 'medico', 'o', 'il', 'farmacista', '.'], ['Per', 'maggiori', 'informazioni', 'riguardo', 'alle', 'motivazioni', 'delle', 'raccomandazioni', 'del', 'CHMP', ',', 'leggere', 'la', 'discussione', 'scientifica', '(', 'anch', 'essa', 'allegato', 'all', 'EPAR', ')', '.'], ['Che', 'cos', 'è', 'Alimta', '?'], ['Alimta', 'è', 'una', 

In [190]:
print(tgt_sents[92])

['Vengono', 'esaminati', 'anche', 'i', 'punti', 'di', 'vista', 'delle', 'parti', 'sociali', ',', 'dedicando', "un'", 'attenzione', 'particolare', 'alle', 'proposte', 'volte', 'ad', 'eliminare', 'le', 'barriere', 'inerenti', "all'", 'età', '.']


In [191]:
# do the alignment
aligns = []
for en_s, tgt_s in zip(tqdm(en_sents), tgt_sents):
    # print(en_s,tgt_s,'\n')
    alignments = aligner.get_word_aligns(en_s, tgt_s)
    align = ''
    for w1, w2 in alignments[mode]:
        align += f'{w1}-{w2} '
    align = align.strip()
    aligns.append(align)

100%|██████████| 121/121 [01:03<00:00,  1.91it/s]


In [192]:
aligns[0]

'0-0 1-1 2-2 3-3 4-4 5-5 7-11 8-8 9-10 10-7 11-12 12-13 13-14 14-15'

In [193]:
print(len(aligns))
trans_df['English-'+language+' Alignment'] = aligns
trans_df.head()

121


,ID,Sentence,English-Italian Alignment
0,d000.s000,Questo documento è la sintesi di una relazione...,0-0 1-1 2-2 3-3 4-4 5-5 7-11 8-8 9-10 10-7 11-...
1,d000.s001,L EPAR descrive il modo in cui il comitato per...,1-2 2-4 3-7 5-9 6-11 8-12 9-14 10-13 11-15 12-...
2,d000.s002,Per maggiori informazioni riguardanti le propr...,0-0 3-1 4-2 5-3 6-5 7-8 8-6 9-9 11-11 12-12 13...
3,d000.s003,Per maggiori informazioni riguardo alle motiva...,0-0 3-1 4-2 6-4 9-8 10-9 11-7 12-10 13-11 14-1...
4,d000.s004,Che cos è Alimta ?,0-1 1-2 2-3 3-4


In [194]:
trans_df.to_csv(ALIGN_SENT_FILE, sep='\t', index=False)

# ALIGN TOKENS IN TOKEN FILE

In [195]:
tkns_df.head()


,Token ID,Token,Lemma,POS
0,NaN,Questo,NaN,X
1,d000.s000.t000,documento,documento,NOUN
2,NaN,è,essere,VERB
3,NaN,la,NaN,X
4,d000.s000.t001,sintesi,sintesi,NOUN


In [196]:
aligned_tks_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
aligned_tks_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,This,NaN,X,NaN
1,d000.s000.t000,document,document,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...
2,NaN,is,be,VERB,NaN
3,NaN,a,NaN,X,NaN
4,d000.s000.t001,summary,summary,NOUN,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...


In [197]:
aligned_tks_df['Token ID '+tgt_lan] = None
aligned_tks_df['Token '+tgt_lan] = None
aligned_tks_df['Lemma '+tgt_lan] = None
aligned_tks_df['POS '+tgt_lan] = None
aligned_tks_df['Sense '+tgt_lan] = None

In [198]:
aligned_tks_df.head()
print(len(aligned_tks_df['Token']))

2759


In [199]:
trans_df.head()

,ID,Sentence,English-Italian Alignment
0,d000.s000,Questo documento è la sintesi di una relazione...,0-0 1-1 2-2 3-3 4-4 5-5 7-11 8-8 9-10 10-7 11-...
1,d000.s001,L EPAR descrive il modo in cui il comitato per...,1-2 2-4 3-7 5-9 6-11 8-12 9-14 10-13 11-15 12-...
2,d000.s002,Per maggiori informazioni riguardanti le propr...,0-0 3-1 4-2 5-3 6-5 7-8 8-6 9-9 11-11 12-12 13...
3,d000.s003,Per maggiori informazioni riguardo alle motiva...,0-0 3-1 4-2 6-4 9-8 10-9 11-7 12-10 13-11 14-1...
4,d000.s004,Che cos è Alimta ?,0-1 1-2 2-3 3-4


In [200]:
def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

In [201]:
count_en = 0
sentence_num = 0
count_es = 0
for i, row in trans_df.iterrows():
    # print(row.iloc[0])
    sent_i = row.iloc[0]
    # print(sent_i)
    # print(sent_i)
    tgt_sent_i = sent_i
    # print(row)
    # sent_i = sent_i.replace("i","d")
    tgt_sent = tkns_df.loc[count_es:count_es+len(tgt_sents[sentence_num])-1]
    en_sent = aligned_tks_df.loc[count_en:count_en+len(en_sents[sentence_num])-1]
    # print(tgt_sent.loc[2])
    for alignment in row['English-'+language+' Alignment'].split():
        w1, w2 = alignment.split('-')
        row_tgt = tgt_sent.loc[count_es+int(w2)]
        row_en = en_sent.loc[count_en+int(w1)]
        if row_en['Token ID '+tgt_lan] is not None:
            if str(aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan]) != "nan":
                aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] += '➕' + str(row_tgt['Token ID'])
            else:
                aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] = str(row_tgt['Token ID'])
            aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] += '➕' + row_tgt['Token']
            if aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] != process_nan(row_tgt['POS']):
                aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] += '➕' + process_nan(row_tgt['POS'])
            if aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] != process_nan(row_en['BN Synset']):
                if str(aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan]) != "nan":
                    aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] += '➕' + process_nan(row_en['BN Synset'])
        else:
            aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] = row_tgt['Token ID']
            aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] = row_tgt['Token']
            aligned_tks_df.loc[count_en+int(w1), 'Lemma '+tgt_lan] = row_tgt["Lemma"]
            aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] = row_tgt['POS']
            aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] = row_en['BN Synset']

    count_en += len(en_sents[sentence_num])+1
    count_es += len(tgt_sents[sentence_num])+1
    sentence_num += 1
    # print(count_en)

In [202]:
# aligned_tks_df[]
aligned_tks_df.head(30)

,Token ID,Token,Lemma,POS,BN Synset,Token ID IT,Token IT,Lemma IT,POS IT,Sense IT
0,NaN,This,NaN,X,NaN,NaN,Questo,NaN,X,NaN
1,d000.s000.t000,document,document,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...,d000.s000.t000,documento,documento,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...
2,NaN,is,be,VERB,NaN,NaN,è,essere,VERB,NaN
3,NaN,a,NaN,X,NaN,NaN,la,NaN,X,NaN
4,d000.s000.t001,summary,summary,NOUN,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...,d000.s000.t001,sintesi,sintesi,NOUN,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...
5,NaN,of,NaN,X,NaN,NaN,di,NaN,X,NaN
6,NaN,the,NaN,X,NaN,None,None,None,None,None
7,d000.s000.t002,European,european,ADJ,bn:00102440a;wn:european%3:01:00::,d000.s000.t005,europea,europeo,ADJ,bn:00102440a;wn:european%3:01:00::
8,d000.s000.t003,Public,public,ADJ,bn:00109211a;wn:public%3:00:00::,d000.s000.t003,pubblica,pubblico,ADJ,bn:00109211a;wn:public%3:00:00::
9,d000.s000.t004,Assessment,assessment,NOUN,bn:00006502n;wn:assessment%1:04:00::;wn:judgem...,d000.s000.t004,valutazione,valutazione,NOUN,bn:00006502n;wn:assessment%1:04:00::;wn:judgem...


In [203]:
aligned_tks_df.to_csv(ALIGN_TOKEN_FILE, sep='\t', index=False)